In [218]:
import sklearn
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import certifi
import ssl
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import precision_score


from sklearn.linear_model import LogisticRegression
os.environ['SSL_CERT_FILE'] = '/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/certifi/cacert.pem'

In [219]:
url = "https://raw.githubusercontent.com/5ah/Football-Predictor/main/results.csv"
matches_df = pd.read_csv(url, sep=',')
url2 = "https://raw.githubusercontent.com/5ah/Football-Predictor/main/countries.csv"
countries_df = pd.read_csv(url2, sep=',') 
url3 = "https://raw.githubusercontent.com/5ah/Football-Predictor/main/shootouts.csv"
shootouts_df = pd.read_csv(url3, sep=',') 

In [220]:
matches_df

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,False
1,1873-03-08,England,Scotland,4,2,Friendly,London,England,False
2,1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland,False
3,1875-03-06,England,Scotland,2,2,Friendly,London,England,False
4,1876-03-04,Scotland,England,3,0,Friendly,Glasgow,Scotland,False
...,...,...,...,...,...,...,...,...,...
45095,2023-10-17,Lithuania,Hungary,2,2,UEFA Euro qualification,Kaunas,Lithuania,False
45096,2023-10-17,Finland,Kazakhstan,1,2,UEFA Euro qualification,Helsinki,Finland,False
45097,2023-10-17,Northern Ireland,Slovenia,0,1,UEFA Euro qualification,Belfast,Northern Ireland,False
45098,2023-10-17,San Marino,Denmark,1,2,UEFA Euro qualification,Serravalle,San Marino,False


In [221]:
countries_df

,Country
0,Argentina
1,France
2,Brazil
3,England
4,Belgium
5,Portugal
6,Netherlands
7,Spain
8,Italy
9,Croatia


In [222]:
shootouts_df.head()

,date,home_team,away_team,winner
0,1967-08-22,India,Taiwan,Taiwan
1,1971-11-14,South Korea,Vietnam Republic,South Korea
2,1972-05-07,South Korea,Iraq,Iraq
3,1972-05-17,Thailand,South Korea,South Korea
4,1972-05-19,Thailand,Cambodia,Thailand


In [223]:
valid_countries = countries_df['Country'].tolist()
matches = matches_df[(matches_df['home_team'].isin(valid_countries)) | (matches_df['away_team'].isin(valid_countries))]
matches

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
0,1872-11-30,Scotland,England,0,0,Friendly,Glasgow,Scotland,False
1,1873-03-08,England,Scotland,4,2,Friendly,London,England,False
2,1874-03-07,Scotland,England,2,1,Friendly,Glasgow,Scotland,False
3,1875-03-06,England,Scotland,2,2,Friendly,London,England,False
4,1876-03-04,Scotland,England,3,0,Friendly,Glasgow,Scotland,False
...,...,...,...,...,...,...,...,...,...
45092,2023-10-17,England,Italy,3,1,UEFA Euro qualification,London,England,False
45093,2023-10-17,Malta,Ukraine,1,3,UEFA Euro qualification,Ta' Qali,Malta,False
45094,2023-10-17,Serbia,Montenegro,3,1,UEFA Euro qualification,Belgrade,Serbia,False
45095,2023-10-17,Lithuania,Hungary,2,2,UEFA Euro qualification,Kaunas,Lithuania,False


In [224]:
# Convert 'date' to datetime format
matches['date'] = pd.to_datetime(matches['date'])

# Filter the DataFrame for matches from 2014 and onwards
matches = matches[matches['date'].dt.year >= 2014]

matches

/var/folders/wz/jrjrb_q94hl1y39g96yvv9h40000gn/T/ipykernel_78989/4084107380.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matches['date'] = pd.to_datetime(matches['date'])


,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral
36189,2014-01-04,Nigeria,Ethiopia,2,1,Friendly,Abuja,Nigeria,False
36194,2014-01-11,Mali,Nigeria,2,1,African Nations Championship,Cape Town,South Africa,True
36197,2014-01-12,Zimbabwe,Morocco,0,0,African Nations Championship,Cape Town,South Africa,True
36202,2014-01-15,Moldova,Norway,1,2,Friendly,Abu Dhabi,United Arab Emirates,True
36203,2014-01-15,Nigeria,Mozambique,4,2,African Nations Championship,Cape Town,South Africa,True
...,...,...,...,...,...,...,...,...,...
45092,2023-10-17,England,Italy,3,1,UEFA Euro qualification,London,England,False
45093,2023-10-17,Malta,Ukraine,1,3,UEFA Euro qualification,Ta' Qali,Malta,False
45094,2023-10-17,Serbia,Montenegro,3,1,UEFA Euro qualification,Belgrade,Serbia,False
45095,2023-10-17,Lithuania,Hungary,2,2,UEFA Euro qualification,Kaunas,Lithuania,False


In [225]:
matches['date'] = pd.to_datetime(matches['date'])
shootouts_df['date'] = pd.to_datetime(shootouts_df['date'])

# Merge the DataFrames on common columns
merged_df = pd.merge(matches, shootouts_df, on=['date', 'home_team', 'away_team'], how='left')

# Function to determine the winner
def determine_winner(row):
    if row['home_score'] == row['away_score']:  # Check if the match is a tie
        return row['winner'] if pd.notnull(row['winner']) else 'TIE'
    else:
        # Determine the winner based on scores
        return row['home_team'] if row['home_score'] > row['away_score'] else row['away_team']

# Apply the function to each row
merged_df['winner'] = merged_df.apply(determine_winner, axis=1)
matches = merged_df
matches

/var/folders/wz/jrjrb_q94hl1y39g96yvv9h40000gn/T/ipykernel_78989/1809638507.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  matches['date'] = pd.to_datetime(matches['date'])


,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,winner
0,2014-01-04,Nigeria,Ethiopia,2,1,Friendly,Abuja,Nigeria,False,Nigeria
1,2014-01-11,Mali,Nigeria,2,1,African Nations Championship,Cape Town,South Africa,True,Mali
2,2014-01-12,Zimbabwe,Morocco,0,0,African Nations Championship,Cape Town,South Africa,True,TIE
3,2014-01-15,Moldova,Norway,1,2,Friendly,Abu Dhabi,United Arab Emirates,True,Norway
4,2014-01-15,Nigeria,Mozambique,4,2,African Nations Championship,Cape Town,South Africa,True,Nigeria
...,...,...,...,...,...,...,...,...,...,...
4075,2023-10-17,England,Italy,3,1,UEFA Euro qualification,London,England,False,England
4076,2023-10-17,Malta,Ukraine,1,3,UEFA Euro qualification,Ta' Qali,Malta,False,Ukraine
4077,2023-10-17,Serbia,Montenegro,3,1,UEFA Euro qualification,Belgrade,Serbia,False,Serbia
4078,2023-10-17,Lithuania,Hungary,2,2,UEFA Euro qualification,Kaunas,Lithuania,False,TIE


In [226]:
desired_date = '2022-12-18'

# Select the row with the specified date
selected_row = matches.loc[merged_df['date'] == desired_date]

# Print or use the selected row as needed
selected_row

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,winner
3771,2022-12-18,Argentina,France,3,3,FIFA World Cup,Lusail,Qatar,True,Argentina


In [227]:
countries_df

,Country
0,Argentina
1,France
2,Brazil
3,England
4,Belgium
5,Portugal
6,Netherlands
7,Spain
8,Italy
9,Croatia


In [228]:
def calculate_stats(team, results_df, countries_df, recent_games_count=5):
    # Filter the results for the specified team
    team_games = results_df[(results_df['home_team'] == team) | (results_df['away_team'] == team)]
    
    # Total games played
    total_games = len(team_games)
    
    # Wins are counted directly from the 'winner' column
    wins = sum(team_games['winner'] == team)
    
    # Calculate win rate
    win_rate = wins / total_games if total_games > 0 else 0
    
    # Get the rank of the team based on its position in countries_df
    team_rank = countries_df.index[countries_df['Country'] == team].tolist()[0]
    
    # Initialize the count for wins against higher-ranked teams
    higher_ranked_wins = 0
    
    # Goals scored and conceded
    goals_scored = 0
    goals_conceded = 0
    
    # Clean sheets count
    clean_sheets = 0
    
    for index, row in team_games.iterrows():
        # Determine if the team is playing at home or away
        home = row['home_team'] == team
        
        # Update goals scored and conceded
        if home:
            goals_scored += row['home_score']
            goals_conceded += row['away_score']
            if row['away_score'] == 0:
                clean_sheets += 1
        else:
            goals_scored += row['away_score']
            goals_conceded += row['home_score']
            if row['home_score'] == 0:
                clean_sheets += 1
        
        # Check if the opponent is a higher-ranked team and if the team won
        opponent = row['away_team'] if home else row['home_team']
        if opponent in countries_df['Country'].values:
            opponent_rank = countries_df.index[countries_df['Country'] == opponent].tolist()[0]
            if row['winner'] == team and opponent_rank < team_rank:
                higher_ranked_wins += 1
    
    # Calculate statistics
    average_goal_differential = (goals_scored - goals_conceded) / total_games if total_games > 0 else 0
    clean_sheets_rate = clean_sheets / total_games if total_games > 0 else 0
    performance_against_higher_ranked = higher_ranked_wins / total_games if total_games > 0 else 0
    average_goals_scored = goals_scored / total_games if total_games > 0 else 0
    average_goals_conceded = goals_conceded / total_games if total_games > 0 else 0
    recent_form = sum(team_games.tail(recent_games_count)['winner'] == team) / recent_games_count if recent_games_count > 0 else 0

    # Compile statistics into a dictionary
    stats = {
        "Win Rate": win_rate,
        "Average Goal Differential": average_goal_differential,
        "Clean Sheets Rate": clean_sheets_rate,
        "Performance Against Higher Ranked Teams": performance_against_higher_ranked,
        "Average Goals Scored": average_goals_scored,
        "Average Goals Conceded": average_goals_conceded,
        "Recent Form": recent_form
    }
    
    return stats


# Calculate the stats for each team in the countries list
stats = {team: calculate_stats(team, matches, countries_df) for team in countries_df['Country']}

# Convert the stats to a DataFrame
stats_df = pd.DataFrame.from_dict(stats, orient='index')
stats_df.reset_index(inplace=True)
stats_df.rename(columns={'index': 'Country'}, inplace=True)

# Round numeric columns to 5 decimal points
numeric_columns = stats_df.select_dtypes(include=['float64']).columns
stats_df[numeric_columns] = stats_df[numeric_columns].round(5)

# Add a column for row number
stats_df['Rank'] = stats_df.index + 1  # Add 1 to start numbering from 1

# Merge the stats with the countries DataFrame
countries_stats_df = pd.merge(countries_df, stats_df, on='Country', how='left')
countries_stats_df

,Country,Win Rate,Average Goal Differential,Clean Sheets Rate,Performance Against Higher Ranked Teams,Average Goals Scored,Average Goals Conceded,Recent Form,Rank
0,Argentina,0.67460,1.26190,0.57937,0.00000,1.96032,0.69841,1.0,1
1,France,0.68217,1.26357,0.44186,0.00775,2.09302,0.82946,0.8,2
2,Brazil,0.71311,1.51639,0.56557,0.04098,2.10656,0.59016,0.4,3
3,England,0.64754,1.41803,0.54918,0.00820,2.10656,0.68852,0.8,4
4,Belgium,0.70833,1.62500,0.44167,0.04167,2.48333,0.85833,0.8,5
5,Portugal,0.64000,1.44000,0.51200,0.02400,2.18400,0.74400,1.0,6
6,Netherlands,0.57522,0.96460,0.35398,0.06195,2.03540,1.07080,0.6,7
7,Spain,0.63248,1.51282,0.49573,0.05128,2.25641,0.74359,1.0,8
8,Italy,0.57391,0.97391,0.45217,0.10435,1.77391,0.80000,0.6,9
9,Croatia,0.55000,0.69167,0.36667,0.05833,1.72500,1.03333,0.4,10


In [229]:
data = pd.merge(matches, countries_stats_df, how='inner', left_on='home_team', right_on='Country')
data = pd.merge(data, countries_stats_df, how='inner', left_on='away_team', right_on='Country', suffixes=('_home', '_away'))
data

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,winner,...,Rank_home,Country_away,Win Rate_away,Average Goal Differential_away,Clean Sheets Rate_away,Performance Against Higher Ranked Teams_away,Average Goals Scored_away,Average Goals Conceded_away,Recent Form_away,Rank_away
0,2014-05-28,Nigeria,Scotland,2,2,Friendly,London,England,True,TIE,...,40,Scotland,0.48913,0.23913,0.38043,0.08696,1.44565,1.20652,0.4,34
1,2023-06-17,Norway,Scotland,1,2,UEFA Euro qualification,Oslo,Norway,False,Scotland,...,42,Scotland,0.48913,0.23913,0.38043,0.08696,1.44565,1.20652,0.4,34
2,2018-06-02,Mexico,Scotland,1,0,Friendly,Mexico City,Mexico,False,Mexico,...,12,Scotland,0.48913,0.23913,0.38043,0.08696,1.44565,1.20652,0.4,34
3,2021-09-07,Austria,Scotland,0,1,FIFA World Cup qualification,Vienna,Austria,False,Scotland,...,25,Scotland,0.48913,0.23913,0.38043,0.08696,1.44565,1.20652,0.4,34
4,2022-03-29,Austria,Scotland,2,2,Friendly,Vienna,Austria,False,TIE,...,25,Scotland,0.48913,0.23913,0.38043,0.08696,1.44565,1.20652,0.4,34
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1550,2023-09-12,Italy,Ukraine,2,1,UEFA Euro qualification,Milan,Italy,False,Italy,...,9,Ukraine,0.50000,0.39000,0.37000,0.05000,1.42000,1.03000,0.6,22
1551,2021-06-29,Sweden,Ukraine,1,2,UEFA Euro,Glasgow,Scotland,True,Ukraine,...,23,Ukraine,0.50000,0.39000,0.37000,0.05000,1.42000,1.03000,0.6,22
1552,2019-11-17,Serbia,Ukraine,2,2,UEFA Euro qualification,Belgrade,Serbia,False,TIE,...,29,Ukraine,0.50000,0.39000,0.37000,0.05000,1.42000,1.03000,0.6,22
1553,2022-06-01,Scotland,Ukraine,1,3,FIFA World Cup qualification,Glasgow,Scotland,False,Ukraine,...,34,Ukraine,0.50000,0.39000,0.37000,0.05000,1.42000,1.03000,0.6,22


In [230]:
data.isna().sum()

date                                            0
home_team                                       0
away_team                                       0
home_score                                      0
away_score                                      0
tournament                                      0
city                                            0
country                                         0
neutral                                         0
winner                                          0
Country_home                                    0
Win Rate_home                                   0
Average Goal Differential_home                  0
Clean Sheets Rate_home                          0
Performance Against Higher Ranked Teams_home    0
Average Goals Scored_home                       0
Average Goals Conceded_home                     0
Recent Form_home                                0
Rank_home                                       0
Country_away                                    0


# Map winner to binary outcome: 1 for home team win, 0 for away team win or draw
data['winner_binary'] = data.apply(
    lambda row: 1 if row['winner'] == row['home_team'] else 0, axis=1
)


In [231]:
# Map winner to binary outcome: 1 for home team win, 0 for away team win or tie
data['winner_binary'] = data.apply(
    lambda row: 1 if row['winner'] == row['home_team'] else 0, axis=1
)


In [232]:
data

,date,home_team,away_team,home_score,away_score,tournament,city,country,neutral,winner,...,Country_away,Win Rate_away,Average Goal Differential_away,Clean Sheets Rate_away,Performance Against Higher Ranked Teams_away,Average Goals Scored_away,Average Goals Conceded_away,Recent Form_away,Rank_away,winner_binary
0,2014-05-28,Nigeria,Scotland,2,2,Friendly,London,England,True,TIE,...,Scotland,0.48913,0.23913,0.38043,0.08696,1.44565,1.20652,0.4,34,0
1,2023-06-17,Norway,Scotland,1,2,UEFA Euro qualification,Oslo,Norway,False,Scotland,...,Scotland,0.48913,0.23913,0.38043,0.08696,1.44565,1.20652,0.4,34,0
2,2018-06-02,Mexico,Scotland,1,0,Friendly,Mexico City,Mexico,False,Mexico,...,Scotland,0.48913,0.23913,0.38043,0.08696,1.44565,1.20652,0.4,34,1
3,2021-09-07,Austria,Scotland,0,1,FIFA World Cup qualification,Vienna,Austria,False,Scotland,...,Scotland,0.48913,0.23913,0.38043,0.08696,1.44565,1.20652,0.4,34,0
4,2022-03-29,Austria,Scotland,2,2,Friendly,Vienna,Austria,False,TIE,...,Scotland,0.48913,0.23913,0.38043,0.08696,1.44565,1.20652,0.4,34,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1550,2023-09-12,Italy,Ukraine,2,1,UEFA Euro qualification,Milan,Italy,False,Italy,...,Ukraine,0.50000,0.39000,0.37000,0.05000,1.42000,1.03000,0.6,22,1
1551,2021-06-29,Sweden,Ukraine,1,2,UEFA Euro,Glasgow,Scotland,True,Ukraine,...,Ukraine,0.50000,0.39000,0.37000,0.05000,1.42000,1.03000,0.6,22,0
1552,2019-11-17,Serbia,Ukraine,2,2,UEFA Euro qualification,Belgrade,Serbia,False,TIE,...,Ukraine,0.50000,0.39000,0.37000,0.05000,1.42000,1.03000,0.6,22,0
1553,2022-06-01,Scotland,Ukraine,1,3,FIFA World Cup qualification,Glasgow,Scotland,False,Ukraine,...,Ukraine,0.50000,0.39000,0.37000,0.05000,1.42000,1.03000,0.6,22,0


In [233]:
features = [
    'Win Rate_home', 'Average Goal Differential_home', 'Clean Sheets Rate_home', 'Performance Against Higher Ranked Teams_home', 'Average Goals Scored_home', 'Average Goals Conceded_home',
    'Win Rate_away', 'Average Goal Differential_away', 'Clean Sheets Rate_away', 'Performance Against Higher Ranked Teams_away', 'Average Goals Scored_away', 'Average Goals Conceded_away'
]
target = 'winner_binary'

In [234]:
X = data[features]
y = data[target]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [235]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [236]:
svc = SVC(kernel='linear')
svc.fit(X_train_scaled, y_train)

SVC(kernel='linear')

In [237]:
y_pred = svc.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)
print("Accuracy:", accuracy)
print("Classification Report:")
print(report)


Accuracy: 0.5980707395498392
Classification Report:
              precision    recall  f1-score   support

           0       0.56      0.76      0.65       151
           1       0.66      0.44      0.53       160

    accuracy                           0.60       311
   macro avg       0.61      0.60      0.59       311
weighted avg       0.62      0.60      0.59       311



In [238]:


precision_score(y_test, y_pred)

0.6635514018691588

In [239]:
lr = LogisticRegression()
lr.fit(X_train_scaled, y_train)

LogisticRegression()

In [240]:
y_pred = lr.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)
print("Accuracy:", accuracy)
print("Classification Report:")
print(report)

Accuracy: 0.6012861736334405
Classification Report:
              precision    recall  f1-score   support

           0       0.56      0.81      0.66       151
           1       0.69      0.41      0.51       160

    accuracy                           0.60       311
   macro avg       0.63      0.61      0.59       311
weighted avg       0.63      0.60      0.59       311



In [241]:
precision_score(y_test, y_pred)

0.6914893617021277

In [242]:
model_knn = KNeighborsClassifier()
model_knn.fit(X_train_scaled, y_train)

KNeighborsClassifier()

In [243]:
y_pred = model_knn.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)
print("Accuracy:", accuracy)
print("Classification Report:")
print(report)

Accuracy: 0.5755627009646302
Classification Report:
              precision    recall  f1-score   support

           0       0.55      0.68      0.61       151
           1       0.61      0.47      0.54       160

    accuracy                           0.58       311
   macro avg       0.58      0.58      0.57       311
weighted avg       0.58      0.58      0.57       311



In [244]:
precision_score(y_test, y_pred)

0.6129032258064516

In [247]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier

# Function to create a matchup DataFrame based on the provided teams and the stats DataFrame
def create_matchup(team_home, team_away, stats_df):
    # Get the statistics for the home team
    team_home_stats_df = stats_df[stats_df['Country'] == team_home]
    team_away_stats_df = stats_df[stats_df['Country'] == team_away]
    
    # Check if the DataFrames are empty and handle the case where the team is not found
    if team_home_stats_df.empty or team_away_stats_df.empty:
        raise ValueError(f"Team data not found. Ensure that team names are spelled correctly. Team Home: {team_home}, Team Away: {team_away}")

    # Extract the first row as the team's stats
    team_home_stats = team_home_stats_df.iloc[0]
    team_away_stats = team_away_stats_df.iloc[0]

    # Create a DataFrame for the match with the selected features
    matchup_data = pd.DataFrame({
        'Win Rate_home': [team_home_stats['Win Rate']],
        'Average Goal Differential_home': [team_home_stats['Average Goal Differential']],
        'Clean Sheets Rate_home': [team_home_stats['Clean Sheets Rate']],
        'Performance Against Higher Ranked Teams_home': [team_home_stats['Performance Against Higher Ranked Teams']],
        'Average Goals Scored_home': [team_home_stats['Average Goals Scored']],
        'Average Goals Conceded_home': [team_home_stats['Average Goals Conceded']],
        'Recent Form_home': [team_home_stats['Recent Form']],
        'Win Rate_away': [team_away_stats['Win Rate']],
        'Average Goal Differential_away': [team_away_stats['Average Goal Differential']],
        'Clean Sheets Rate_away': [team_away_stats['Clean Sheets Rate']],
        'Performance Against Higher Ranked Teams_away': [team_away_stats['Performance Against Higher Ranked Teams']],
        'Average Goals Scored_away': [team_away_stats['Average Goals Scored']],
        'Average Goals Conceded_away': [team_away_stats['Average Goals Conceded']],
        'Recent Form_away': [team_away_stats['Recent Form']]
    })
    
    return matchup_data

def interpret_prediction(home_team, away_team, prediction):
    if prediction == 1:
        return home_team
    else:
        return away_team  # Or 'Draw' if you want to treat 0 as a draw

def print_previous_matchups(data, team1, team2):
    # Filter the dataset for matches where the two teams faced each other
    matchups = data[((data['home_team'] == team1) & (data['away_team'] == team2)) |
                    ((data['home_team'] == team2) & (data['away_team'] == team1))]

    # Check if there are any previous matchups
    if matchups.empty:
        print(f"No previous matchups found between {team1} and {team2}.")
    else:
        print(f"Previous matchups between {team1} and {team2}:")
        print(matchups)

# Example usage



# Make sure these feature names match exactly with your dataframe columns
features = [
    'Win Rate_home', 'Average Goal Differential_home', 'Clean Sheets Rate_home',
    'Performance Against Higher Ranked Teams_home', 'Average Goals Scored_home',
    'Average Goals Conceded_home', 'Recent Form_home', 'Win Rate_away',
    'Average Goal Differential_away', 'Clean Sheets Rate_away',
    'Performance Against Higher Ranked Teams_away', 'Average Goals Scored_away',
    'Average Goals Conceded_away', 'Recent Form_away'
]
target = 'winner_binary'  # This should be the name of your target column

# Splitting the data into features (X) and target (y)
X = data[features]
y = data[target]

# Load your pre-trained models (replace 'your_pretrained_model' with the actual models)
logistic_model = LogisticRegression()  # replace with your pre-trained model
svc_model = SVC()  # replace with your pre-trained model
knn_model = KNeighborsClassifier()  # replace with your pre-trained model

# Create the matchup
team_A = 'Wales'
team_B = 'Serbia'
matchup = create_matchup(team_A, team_B, countries_stats_df)
print_previous_matchups(matches, team_A, team_B)

# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Feature scaling
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Logistic Regression
logistic_model = LogisticRegression()
logistic_model.fit(X_train_scaled, y_train)

# Support Vector Classifier
svc_model = SVC()
svc_model.fit(X_train_scaled, y_train)

# K-Nearest Neighbors
knn_model = KNeighborsClassifier()
knn_model.fit(X_train_scaled, y_train)

# Feature scaling (if your model was trained with scaled features)
scaler = StandardScaler()
matchup_scaled = scaler.fit_transform(matchup)

# Make predictions
logistic_prediction = logistic_model.predict(matchup_scaled)[0]  # [0] to get the first element of the prediction
svc_prediction = svc_model.predict(matchup_scaled)[0]
knn_prediction = knn_model.predict(matchup_scaled)[0]

# Print the predicted winners
print('Logistic Regression Predicted Winner:', interpret_prediction(team_A, team_B, logistic_prediction))
print('Support Vector Classification Predicted Winner:', interpret_prediction(team_A, team_B, svc_prediction))
print('K-Nearest Neighbors Predicted Winner:', interpret_prediction(team_A, team_B, knn_prediction))


Previous matchups between Wales and Serbia:
           date home_team away_team  home_score  away_score  \
1239 2016-11-12     Wales    Serbia           1           1   
1437 2017-06-11    Serbia     Wales           1           1   

                        tournament      city country  neutral winner  
1239  FIFA World Cup qualification   Cardiff   Wales    False    TIE  
1437  FIFA World Cup qualification  Belgrade  Serbia    False    TIE  
Logistic Regression Predicted Winner: Serbia
Support Vector Classification Predicted Winner: Serbia
K-Nearest Neighbors Predicted Winner: Serbia
